### Import Libraries

In [ ]:
!pip install ibm_watson

In [ ]:
!pip install pandas

In [ ]:
!pip install matplotlib

In [ ]:
!pip install langdetect

In [ ]:
!pip install ibm_db

In [ ]:
import ibm_db
import pandas as pd
import numpy
import matplotlib.pyplot as plt
import json
from langdetect import detect
import dateutil.parser

## Insert The Required Credentials Here

1. Insert Watson Natural Language Understanding API key and URL here

In [ ]:
# Insert Watson Natural Language Understanding API key and URL here

watson_nlu_api_key = '<YOUR_NLU_API_KEY>'
watson_nlu_url = '<YOUR_NLU_URL>'

2. Insert credentials to load Customer Review Dataset here

In [ ]:
# Insert Customer Review Dataset here


3. Insert credentials to load Customer Order Details Dataset here

In [ ]:
# Insert Customer Order Details Dataset here


4. Insert Db2 Connection Credentials here

In [ ]:
# Insert Db2 Connection Credentials here


## Setup Watson NLU

In [ ]:
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_watson.natural_language_understanding_v1 \
    import Features, EntitiesOptions, KeywordsOptions, SyntaxOptions, SyntaxOptionsTokens
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [ ]:
authenticator = IAMAuthenticator(watson_nlu_api_key)
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2019-07-12',
    authenticator=authenticator
)

In [ ]:
natural_language_understanding.set_service_url(watson_nlu_url)

### Understanding the data better

In [ ]:
data.head(5)

In [ ]:
data.shape

### Summarize the data

In [ ]:
data.describe()

#### There are 65 different Customers

In [ ]:
uniqueCustomers = data['id'].unique()
len(uniqueCustomers)

#### We select five product review

In [ ]:
reviewBySingleCustomer = data.loc[data['id'] == 'AVpgNzjwLJeJML43Kpxn']
productId1 = reviewBySingleCustomer[:527]
productId1.describe()

In [ ]:
reviewBySingleCustomer = data.loc[data['id'] == 'AVpe7xlELJeJML43ypLz']
productId2 = reviewBySingleCustomer[:392]
productId2.describe()

In [ ]:
reviewBySingleCustomer = data.loc[data['id'] == 'AVqVGWQDv8e3D1O-ldFr']
productId3 = reviewBySingleCustomer[:388]
productId3.describe()

In [ ]:
reviewBySingleCustomer = data.loc[data['id'] == 'AVqkIhxunnc1JgDc3kg_']
productId4 = reviewBySingleCustomer[:488]
productId4.describe()

In [ ]:
reviewBySingleCustomer = data.loc[data['id'] == 'AVpfw2hvilAPnD_xh0rH']
productId5 = reviewBySingleCustomer[:484]
productId5.describe()

In [ ]:
uniqueProductReviews = pd.concat([productId1, productId2, productId3, productId4, productId5])

#### The customer reviews  for five products

In [ ]:
uniqueProductReviews.head()

#### has 2279 records

In [ ]:
uniqueProductReviews.shape

In [ ]:
uniqueProductReviews.describe()

In [ ]:
len(uniqueProductReviews['id'].unique())

### We then Pass the reviews from dataset one by one to get sentiments and emotions

In [ ]:
sellerScores = []
emotionScores = []
for texts in uniqueProductReviews['reviews.text']:
    
    try:
        language = detect(texts)
    except:
        continue
        
    if language != 'en':
        continue
    
    wordLength = len(texts.split()) 
    print("\n"+texts+"\n", wordLength, sep="\t")
    
    if int(wordLength) < 5:
        continue
    
    response = natural_language_understanding.analyze(
        text=texts,
        features=Features(keywords=KeywordsOptions(sentiment=True,emotion=True,limit=2))).get_result()
    jsonResponse = json.dumps(response, indent=2)
    
    X = json.loads(jsonResponse)
    
    if not X["keywords"]:
        continue
       
    sentiment = X["keywords"][0]["sentiment"]
    if sentiment['score'] < 0:
        sentiment['score'] = 1 + sentiment['score']
    sentimentScore = sentiment['score']

    emotion = X["keywords"][0]["emotion"]
    emotions = [emotion["sadness"], emotion["joy"], emotion["fear"], emotion["disgust"], emotion["anger"]]

    j = 0
    position = 0
    emo = ''
    for i in emotions:
        if i > j:
            j = i
            index = position
        position += 1

    if index == 0:
        emo = 'sad'
    if index == 1:
        emo = 'joy'
    if index == 2:
        emo = 'fear'
    if index == 3:
        emo = 'disgust'
    if index == 4:
        emo = 'anger'

    emotionScore = j

    AveragedSum = (sentimentScore + emotionScore) / 2

    sellerScores.append(AveragedSum)
    emotionScores.append(emo)

#### You can see scores and and emotions for 2124 records out of 2279 

This is because there were 155 records which were not understood by NLU and hence NLU had to discard them

In [ ]:
print(sellerScores, emotionScores, sep = "\n\n")
print(len(sellerScores), len(emotionScores), sep = "\n\n")

### Understanding the seller scores better

In [ ]:
sellerScoreDf = pd.DataFrame(sellerScores, columns=["sellerScore"])

### Rate the seller between 1 to 3

##### We Split 3 based on 25% (0.34), 50% (0.60) and 75% (0.81) 

In [ ]:
sellerScoreDf.describe()

In [ ]:
satisfactionScore = []
minimum = 0.022058
maximum = 0.991763
q1 = 0.348248
q2 = 0.601153
q3 = 0.811665

for individualScore in sellerScoreDf['sellerScore']:
    
    if individualScore >= minimum and individualScore <= q1:
        satisfactionScore.append(0)
    
    if individualScore > q1 and individualScore <= q2:
        satisfactionScore.append(1)
    
    if individualScore > q2 and individualScore <= q3:
        satisfactionScore.append(2)
    
    if individualScore > q3 and individualScore <= maximum:
        satisfactionScore.append(3)


In [ ]:
print(len(satisfactionScore))

In [ ]:
ids = []
for items in uniqueProductReviews['id']:
    ids.append(items)
len(ids)

In [ ]:
for i in range(0, 155, 1):
    ids.pop(i)

In [ ]:
len(ids)

### We create the first dataset for our use, that is Seller Satisfaction Scores based on Customer Reviews.

In [ ]:
sellerScoreDf.insert(0, 'id', ids)

In [ ]:
sellerScoreDf.insert(1, 'emotion', emotionScores)

In [ ]:
sellerScoreDf.insert(2, 'Customer_Review_Satisfaction', satisfactionScore)

In [ ]:
dataset1 = sellerScoreDf
dataset1.head()

In [ ]:
dataset1.to_csv('Customer_Review_Satisfaction.csv', encoding='utf-8', index=False)

### We Can Visualize the dataset

In [ ]:
viz = sellerScoreDf[['sellerScore','Customer_Review_Satisfaction']]
viz.hist()
plt.show()

In [ ]:
plt.scatter(sellerScoreDf.Customer_Review_Satisfaction, sellerScoreDf.sellerScore,  color='blue')
plt.xlabel("Seller Score Avg")
plt.ylabel("Seller Satisfaction score (out of 3)")
plt.show()

### Import Dataset

2. Customer orders Dataset

In [ ]:
data2 = data2[['order_id', 'product_id' ,'order_status', 'order_estimated_delivery_date', 'order_delivered_customer_date']]
data2.head()

### Understanding the Dataset better

In [ ]:
data2.shape

In [ ]:
data2.describe()

#### There are total 7 types of order status

In [ ]:
uniqueOrderStatus = data2['order_status'].unique()
print(uniqueOrderStatus)

In [ ]:
OrderStatus = []
for stat in uniqueOrderStatus:
    orderBySingleCustomerStatus = data2.loc[data2['order_status'] == stat]
    OrderStatus.append(len(orderBySingleCustomerStatus))

#### Breakdown of the order status
'delivered' -> 110197
'invoiced' -> 359
'shipped' -> 1185
'canceled' -> 542
'processing' -> 357
'unavailable' -> 7
'approved' -> 3

In [ ]:
print(OrderStatus)

#### We select five products order details

In [ ]:
orderBySingleCustomerStatus1 = data2.loc[data2['product_id'] == 'aca2eb7d00ea1a7b8ebd4e68314663af']
productId1 = orderBySingleCustomerStatus1
productId1.describe()

In [ ]:
orderBySingleCustomerStatus1 = data2.loc[data2['product_id'] == '389d119b48cf3043d311335e499d9c6b']
productId2 = orderBySingleCustomerStatus1
productId2.describe()

In [ ]:
orderBySingleCustomerStatus1 = data2.loc[data2['product_id'] == '368c6c730842d78016ad823897a372db']
productId3 = orderBySingleCustomerStatus1
productId3.describe()

In [ ]:
orderBySingleCustomerStatus1 = data2.loc[data2['product_id'] == '99a4788cb24856965c36a24e339b6058']
productId4 = orderBySingleCustomerStatus1
productId4.describe()

In [ ]:
orderBySingleCustomerStatus1 = data2.loc[data2['product_id'] == '422879e10f46682990de24d770e7f83d']
productId5 = orderBySingleCustomerStatus1
productId5.describe()

In [ ]:
uniqueOrderStatus = pd.concat([productId1, productId2, productId3, productId4, productId5])

In [ ]:
uniqueOrderStatus.describe()

In [ ]:
len(uniqueOrderStatus['product_id'].unique())

#### Refactor `order_estimated_delivery_date`	& `order_delivered_customer_date` columns

In [ ]:
allEstimatedDates = [dates for dates in uniqueOrderStatus['order_estimated_delivery_date']]

In [ ]:
estimatedDates = []
for b in allEstimatedDates:
    d = dateutil.parser.parse(b).date()
    estimatedDates.append(d)

In [ ]:
uniqueOrderStatus.insert(2, 'estimated_date', estimatedDates)
uniqueOrderStatus = uniqueOrderStatus.drop(['order_estimated_delivery_date'], axis=1)

In [ ]:
allDeliveredDates = [dates for dates in uniqueOrderStatus['order_delivered_customer_date']]

In [ ]:
deliveredDates = []
for b in allDeliveredDates:
    try:
        d = dateutil.parser.parse(b).date()
        deliveredDates.append(d)
    except:
        deliveredDates.append(0)

In [ ]:
uniqueOrderStatus.insert(3, 'delivered_date', deliveredDates)
uniqueOrderStatus = uniqueOrderStatus.drop(['order_delivered_customer_date'], axis=1)

In [ ]:
uniqueOrderStatus.head()

In [ ]:
uniqueOrderStatusShuffled = uniqueOrderStatus.sample(frac=1).reset_index(drop=True)
uniqueOrderStatusShuffled.head()

In [ ]:
data2.to_csv('order_details2.csv', encoding='utf-8', index=False)

### Calculate Seller rating out of 2
1. Based on Order Status

In [ ]:
rating = []
for x in uniqueOrderStatusShuffled['order_status']:
    if x == 'delivered':
        rating.append(1)
    if x == 'canceled':
        rating.append(0)
    if x == 'unavailable':
        rating.append(0)
    else:
        rating.append(0)

In [ ]:
delta = []
j = 0
x = uniqueOrderStatusShuffled['estimated_date']
for i in uniqueOrderStatusShuffled['delivered_date']:
    if i == 0:
        delta.append(999)
        j = j+1
    else:
        y = i - x[j]
        delta.append(y.days)
        j =j+1

In [ ]:
rating2 = []
for i in delta:
    if i <= 0:
        rating2.append(1)
    if i > 0:
        rating2.append(0)

In [ ]:
finalRating = [sum(x) for x in zip(rating, rating2)]

In [ ]:
initialRating = [x for x in sellerScoreDf['Customer_Review_Satisfaction']]

### Calculate the seller score by suming up the customer reviews score and order details score

In [ ]:
sellersatisfactionScore = [sum(x) for x in zip(initialRating, finalRating)]

In [ ]:
sellerScoreDf.head()

In [ ]:
len(initialRating)

In [ ]:
len(finalRating)

In [ ]:
len(sellersatisfactionScore)

In [ ]:
len(uniqueOrderStatusShuffled['product_id'])

In [ ]:
ids = []
for items in uniqueProductReviews['id']:
    ids.append(items)
print(len(ids))

for i in range(0, 155, 1):
    ids.pop(i)
print(len(ids))

In [ ]:
emotions = []
for items in sellerScoreDf['emotion']:
    emotions.append(items)
print(len(ids))

In [ ]:
print(len(finalRating))

for i in range(0, 155, 1):
    finalRating.pop(i)
print(len(finalRating))

### Finally Create the Dataset that contains product_ids, customer_emotions, customer_review_satisfaction_score, customer_order_satisfaction_score and seller_quality_score

In [ ]:
secondPartDataFrame = pd.DataFrame(ids, columns=['product_ids'])

In [ ]:
secondPartDataFrame.insert(1, 'customer_emotions', emotions)

In [ ]:
secondPartDataFrame.insert(2, 'customer_review_satisfaction_score', initialRating)

In [ ]:
secondPartDataFrame.insert(3, 'customer_order_satisfaction_score', finalRating)

In [ ]:
secondPartDataFrame.insert(4, 'seller_quality_score', sellersatisfactionScore)

In [ ]:
secondPartDataFrame.head()

In [ ]:
dataset2 = secondPartDataFrame
dataset2.head()

In [ ]:
dataset2.to_csv('New_seller_quality_scores.csv', encoding='utf-8', index=False)

### Insert the dataset into DB2 on Cloud

In [ ]:
dsn_driver = "IBM DB2 ODBC DRIVER"
dsn_database = credentials_1['database'] 
dsn_hostname = credentials_1['host']
dsn_port = "50000"                
dsn_uid = credentials_1['username']      
dsn_pwd = credentials_1['password']

dsn = (My 
    "DRIVER={{IBM DB2 ODBC DRIVER}};"
    "DATABASE={0};"
    "HOSTNAME={1};"
    "PORT={2};"
    "PROTOCOL=TCPIP;"
    "UID={3};"
    "PWD={4};").format(dsn_database, dsn_hostname, dsn_port, dsn_uid, dsn_pwd)

conn = ibm_db.connect(dsn, "", "")

In [ ]:
create_statement = 'CREATE TABLE "SELLERQUALITYSCORE" (\n"product_ids" VARCHAR,\n "customer_emotions" VARCHAR,\n  "customer_review_satisfaction_score" VARCHAR,\n  "customer_order_satisfaction_score" VARCHAR,\n  "seller_quality_score" VARCHAR\n)'
create_statement = create_statement.replace('VARCHAR', 'VARCHAR(256)')
ibm_db.exec_immediate(conn, create_statement)

In [ ]:
for index, row in dataset2.iterrows():

    a ='\''
    product_ids = a+str(row['product_ids'])+a
    customer_emotions = a+str(row['customer_emotions'])+a
    customer_review_satisfaction_score = a+str(row['customer_review_satisfaction_score'])+a
    customer_order_satisfaction_score = a+str(row['customer_order_satisfaction_score'])+a
    seller_quality_score = a+str(row['seller_quality_score'])+a
    
    insert = 'INSERT INTO %s.SELLERQUALITYSCORE VALUES(%s, %s, %s, %s, %s)' %(dsn_uid, product_ids, customer_emotions, customer_review_satisfaction_score, customer_order_satisfaction_score, seller_quality_score)
    ibm_db.exec_immediate(conn, insert)